# Звіт з лабораторної роботи №10

Виконав студент групи КН-24-1 Озівський В. В.

Тема: Стиснення даних. Жадібна стратегія на прикладі кодування Гафмена
\
Мета: навчитись реалізовувати алгоритм побудови дерева оптимальних кодів Гафмена на основі черги з пріоритетом (за допомогою купи) засобами Python.

### <center>Хід роботи</center>

### 1. Побудувати дерево кодів Гафмена згідно з варіантом і візуалізувати дерево. 

#### Коди Гафмена для кожного символу AABBBCCCDDDDDEEEEEFFFFFGGGGGHHHHHIIII:
- A: 1100 (частота: 2)
- B: 000 (частота: 3)
- C: 1101 (частота: 3)
- D: 010 (частота: 5)
- E: 011 (частота: 5)
- F: 101 (частота: 5)
- G: 100 (частота: 5)
- H: 111 (частота: 5)
- I: 001 (частота: 4)

#### Двійкове дерево

![Двійкове дерево](1.png)

#### 2. Перевірити результат за допомогою коду

In [1]:
# A Вузол дерева Гафмена
import heapq


class node:
    def __init__(self, freq, symbol, left=None, right=None):
        # частота символу
        self.freq = freq
        # назва символу (символ)
        self.symbol = symbol
        # вузол ліворуч від поточного вузла
        self.left = left
        # вузол праворуч від поточного вузла
        self.right = right
        # напрямок дерева (0/1)
        self.huff = ''

    def __lt__(self, nxt):
        return self.freq < nxt.freq


# утиліта для виведення кодів Гафмена для
# всіх символів у щойно створеному дереві Гафмена
def printNodes(node, val=''):
    # Код Гафмена для поточного вузла
    newVal = val + str(node.huff)
    # якщо вершина не є реберною вершиною
    # то пройти всередині неї
    if (node.left):
        printNodes(node.left, newVal)
    if (node.right):
        printNodes(node.right, newVal)
        # якщо node є реберною вершиною тоді
        # вивести його хаффманівський код
    if (not node.left and not node.right):
        print(f"{node.symbol} -> {newVal}")


# символи для дерева Гафмена
chars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
# частота символів
freq = [2, 3, 3, 5, 5, 5, 5, 5, 4]
# список, що містить невикористані вершини
nodes = []
# перетворення символів та частот
# у вузли дерева Гафмена
for x in range(len(chars)):
    heapq.heappush(nodes, node(freq[x], chars[x]))
while len(nodes) > 1:
    # відсортувати всі вершини за зростанням
    # на основі їх частоти
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)
    # присвоїти значення напрямку цим вузлам
    left.huff = 0
    right.huff = 1
    # об'єднати 2 найменші вершини, щоб створити
    # новий вузол як їхній батько
    newNode = node(left.freq + right.freq, left.symbol + right.symbol, left, right)
    heapq.heappush(nodes, newNode)

# Друк кодів Гафмена
printNodes(nodes[0])

B -> 000
I -> 001
D -> 010
E -> 011
G -> 100
F -> 101
A -> 1100
C -> 1101
H -> 111


#### 3. Процедура на Python, яка для вхідного повідомлення обчислює список символів chars та список їх частот freq

In [2]:
def get_freq_table(message):
    
    char_freq = {}
    
    for char in message:
        if char in char_freq:
            char_freq[char] += 1
        else:
            char_freq[char] = 1
    
    chars = list(char_freq.keys())
    freq = [char_freq[char] for char in chars]
    
    return chars, freq

#### 4. Декодування повідомлення, заданого варіантом

In [5]:
import heapq
from collections import defaultdict

# to map each character its huffman value
codes = {}

# To store the frequency of character of the input data
freq = defaultdict(int)

# A Huffman tree node
class MinHeapNode:
    def __init__(self, data, freq):
        self.left = None
        self.right = None
        self.data = data
        self.freq = freq

    def __lt__(self, other):
        return self.freq < other.freq

# utility function to print characters along with
# there huffman value
def printCodes(root, str):
    if root is None:
        return
    if root.data != '$':
        print(root.data, ":", str)
    printCodes(root.left, str + "0")
    printCodes(root.right, str + "1")

# utility function to store characters along with
# there huffman value in a hash table
def storeCodes(root, str):
    if root is None:
        return
    if root.data != '$':
        codes[root.data] = str
    storeCodes(root.left, str + "0")
    storeCodes(root.right, str + "1")

# function to build the Huffman tree and store it
# in minHeap
def HuffmanCodes(size):
    global minHeap
    for key in freq:
        minHeap.append(MinHeapNode(key, freq[key]))
    heapq.heapify(minHeap)
    while len(minHeap) != 1:
        left = heapq.heappop(minHeap)
        right = heapq.heappop(minHeap)
        top = MinHeapNode('$', left.freq + right.freq)
        top.left = left
        top.right = right
        heapq.heappush(minHeap, top)
    storeCodes(minHeap[0], "")

# utility function to store map each character with its
# frequency in input string
def calcFreq(str, n):
    for i in range(n):
        freq[str[i]] += 1

# function iterates through the encoded string s
# if s[i]=='1' then move to node->right
# if s[i]=='0' then move to node->left
# if leaf node append the node->data to our output string
def decode_file(root, s):
    ans = ""
    curr = root
    n = len(s)
    for i in range(n):
        if s[i] == '0':
            curr = curr.left
        else:
            curr = curr.right

        # reached leaf node
        if curr.left is None and curr.right is None:
            ans += curr.data
            curr = root
    return ans + '\0'

# Driver code
if __name__ == "__main__":
    minHeap = []
    str = "AABBBCCCDDDDDEEEEEFFFFFGGGGGHHHHHIIII"
    encodedString, decodedString = "", ""
    calcFreq(str, len(str))
    HuffmanCodes(len(str))
    print("Character With there Frequencies:")
    for key in sorted(codes):
        print(key, codes[key])

    for i in str:
        encodedString += codes[i]

    print("\nEncoded Huffman data:")
    print(encodedString)

    # Function call
    decodedString = decode_file(minHeap[0], encodedString)
    print("\nDecoded Huffman Data:")
    print(decodedString)

Character With there Frequencies:
A 1100
B 000
C 1101
D 010
E 011
F 101
G 100
H 111
I 001

Encoded Huffman data:
11001100000000000110111011101010010010010010011011011011011101101101101101100100100100100111111111111111001001001001

Decoded Huffman Data:
AABBBCCCDDDDDEEEEEFFFFFGGGGGHHHHHIIII 


### Відповіді на контрольні питання

1. Що таке жадібні алгоритми?
\
Жадібні алгоритми — це алгоритми, які приймають найкраще локальне рішення на кожному кроці з надією отримати глобально оптимальне рішення.

2. Що таке префіксний код? Який код використовується у коді Гафмена?
\
Префіксний код — це код, у якому жоден код слова не є префіксом іншого. У коді Гафмена використовується саме **префіксний двійковий код**.

3. Як пов’язана структура даних «купа» зі структурою даних «черга з пріоритетами»?
\
Купа (heap) — це ефективна реалізація черги з пріоритетами, яка дозволяє швидко знаходити й видаляти елемент з найвищим (або найнижчим) пріоритетом.

4. Що таке стиснення даних і для чого воно використовується? Які його основні переваги?
\
Стиснення — це процес зменшення розміру даних для економії пам’яті або пропускної здатності. Переваги: менше місця на диску, швидша передача через мережу, пришвидшення обробки.

5. Які кроки необхідно виконати для стиснення даних за допомогою алгоритму кодування Гафмена?
   1. Обчислити частоти символів.
   2. Створити вузли для кожного символу.
   3. Побудувати дерево Гафмена з купою.
   4. Призначити коди символам.
   5. Замінити символи у даних на відповідні коди.


6. Які основні обмеження та недоліки алгоритму кодування Гафмена? Чи можливо покращити його продуктивність?
\
Недоліки: неефективний для коротких повідомлень або символів з майже однаковими частотами. Можна покращити, наприклад, за допомогою адаптивного або арифметичного кодування.

7. Які існують альтернативні методи стиснення даних, що можуть конкурувати з алгоритмом Гафмена?
   - Арифметичне кодування
   - LZ77, LZ78, LZW
   - BWT (Burrows-Wheeler Transform)
   - RLE (Run-Length Encoding)

8. Які практичні застосування можуть мати алгоритми стиснення даних, зокрема алгоритм Гафмена, у сучасних інформаційних системах?
   - Архіватори (ZIP, RAR)
   - Формати файлів (JPEG, MP3, PNG)
   - Передача даних через мережу
   - Бази даних, кешування та системи з обмеженим простором